In [4]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime as d
import time

In [21]:
prev = d.now()
prev = prev.strftime("%d-%m %H:%M")

In [22]:
prev

'22-05 13:14'

In [5]:
lista_server=["BR1","EUW1","JP1","KR","NA1",'OC1']


In [46]:
lista_m

{'status': {'message': 'Service unavailable', 'status_code': 503}}

In [7]:
def actual_ladder(server):

    
    lista_cha=requests.get(f"https://{server}.api.riotgames.com/tft/league/v1/challenger?api_key=RGAPI-68951cc5-0345-4a6e-af85-d9e541ec159c").json()
    lista_gm=requests.get(f"https://{server}.api.riotgames.com/tft/league/v1/grandmaster?api_key=RGAPI-68951cc5-0345-4a6e-af85-d9e541ec159c").json()
    lista_m=requests.get(f"https://{server}.api.riotgames.com/tft/league/v1/master?api_key=RGAPI-68951cc5-0345-4a6e-af85-d9e541ec159c").json()

    for entrie in lista_gm['entries']:
        lista_cha['entries'].append(entrie)
    for entrie in lista_m['entries']:
        lista_cha['entries'].append(entrie)

    nick=[]
    for i in range(len(lista_cha["entries"])):
        nick.append(lista_cha['entries'][i]["summonerName"])

    lp=[]
    for i in range(len(lista_cha["entries"])):
        lp.append(lista_cha['entries'][i]["leaguePoints"])
        
    games=[]
    for i in range(len(lista_cha["entries"])):
        games.append(lista_cha["entries"][i]["wins"]+lista_cha["entries"][i]["losses"])


    df=pd.DataFrame(lp,nick).reset_index().rename(columns={"index":"Nick",0:"League Points"})
    df["Daily Games"]=0
    df["Games"] = games
    df=df.sort_values("League Points",ascending=False).reset_index(drop=True)
    # df.to_csv(f'dia_ant{server}.csv',index=False) 
    # time.sleep(2) 
    return df



In [2]:
def day_ladder(server):
        print(server)
        result=''.join([i for i in server if not i.isdigit()])
        df=actual_ladder(server)
        dia_ant = pd.read_csv(f"dia_ant{server}.csv")
        
        dfa=df.merge(dia_ant,how="left",on="Nick")
        parcial=pd.DataFrame()
        parcial["Nick"]=dfa["Nick"]
        parcial["League Points"]=dfa["League Points_x"]
        parcial["Total Games"]=dfa["Games_x"]
        parcial["Daily League Points"]=dfa["League Points_x"]-dfa["League Points_y"]
        parcial["Daily Games"]=dfa["Games_x"]-dfa["Games_y"]
        parcial=parcial.sort_values("Daily League Points",ascending=False).reset_index(drop=True)
        parcial.sort_values(['Daily League Points', 'League Points'], ascending=[False, False], inplace=True)
        lolchess=[]
        moba=[]
        for nick in parcial.Nick:
                lolchess.append(f"https://lolchess.gg/profile/{result}/{nick}")
                moba.append(f"https://app.mobalytics.gg/pt_br/tft/profile/{result}/{nick}/overview")
        parcial["lolchess"]=lolchess
        parcial["mobalytics"]=moba
        cols=["Nick", "Daily League Points", "Daily Games", "League Points", "Total Games","lolchess", "mobalytics"]




        parcial.to_csv(f"../data/parcial{server}.csv")
        return dfa
        


In [8]:

    for server in lista_server:
        dfa=day_ladder(server)


BR1


FileNotFoundError: [Errno 2] No such file or directory: 'dia_antBR1.csv'

In [6]:
pd.read_csv("../data/dia_antBR1.csv")

,Nick,League Points,Daily Games,Games
0,DETETIVE ALINE,1916,0,285
1,DobZ,1886,0,168
2,BEE itachi,1763,0,299
3,FOUR Jhin,1514,0,392
4,BES DobZ,1497,0,250
...,...,...,...,...
4639,BSerejo,0,0,341
4640,Koala Esbelto,0,0,168
4641,TFT Baume,0,0,411
4642,Insec Nilton,0,0,95


In [48]:
def reset_day(dfa,server):
    dia_ant=pd.read_csv(f'dia_ant{server}.csv')
    dfa=pd.read_csv(f'parcial{server}.csv')
    dia_ant=dfa
    dia_ant = dia_ant.to_csv(f"dia_ant{server}.csv")